# Tema 1 IA - Class Scheduler
### Alexandru LICURICEANU - 332CD

In [ ]:
# Prerequisites
import utils
import itertools
import math
import time
from copy import copy, deepcopy
from heapq import heappop, heappush

from utils import ZILE as DAYS
from utils import INTERVALE as INTERVALS
from utils import SALI as CLASSROOMS
from utils import MATERII as SUBJECTS
from utils import PROFESORI as TEACHERS

CONSTRAINTS = 'Constrangeri'
PREFERRED = 'Preferred'
NOT_PREFERRED = 'Not_preferred'
CAPACITY = 'Capacitate'
MAX_INTERVALS = 7

# A* Algorithm

`State` data structure used by the A* implementation.

In [ ]:
class State:
    def __init__(self, cost, state):
        self.state = state
        self.cost = cost

    def __lt__(self, other):
        return self.cost < other.cost

`init_empty_timetable()` Returns an empty timetable.



In [ ]:
def init_empty_timetable(data):
    timetable = {day: {interval: {classroom: None for classroom in data[CLASSROOMS]} for interval in data[INTERVALS]} for day in data[DAYS]}
    return timetable

`is_covered()` Checks the timetable to see if the subject is fully covered.

In [ ]:
def is_covered(state, subject, data):
    student_count = 0

    # Sum the capacity of all assigned classrooms where the subject is taught. 
    for day in state:
        for interval in state[day]:
            for classroom in state[day][interval]:
                if state[day][interval][classroom] is not None and state[day][interval][classroom][1] == subject:
                    student_count += data[CLASSROOMS][classroom][CAPACITY]

    return student_count >= data[SUBJECTS][subject]

`count_teacher_slots()` Counts the number of slots assigned to a given teacher in the timetable.

In [ ]:
def count_teacher_slots(state, teacher):
    teacher_count = 0

    for day in state:
        for interval in state[day]:
            for classroom in state[day][interval]:
                if state[day][interval][classroom] is not None and state[day][interval][classroom][0] == teacher:
                    teacher_count += 1

    return teacher_count

`choose_teacher()` Chooses the teacher that has the least amount of classed for this subject assigned to him.

In [ ]:
def choose_teacher(state, subject, data):
    teacher_counts = {teacher: 0 for teacher in data[TEACHERS] if subject in data[TEACHERS][teacher][SUBJECTS]}

    for day in state:
        for interval in state[day]:
            for classroom in state[day][interval]:
                if state[day][interval][classroom] is not None and state[day][interval][classroom][1] == subject:
                    teacher_counts[state[day][interval][classroom][0]] += 1

    # Filter out teachers that have already reached the maximum number of classes.
    teacher_counts = {teacher: count for teacher, count in teacher_counts.items() if count_teacher_slots(state, teacher) < MAX_INTERVALS}

    return min(teacher_counts, key=teacher_counts.get)


`check_hard_constraints()` Calculates the hard cost based on how many hard constraints have been violated in the state.

In [ ]:
def check_hard_constraints(state, data):
    hard_cost = 0
    
    # Check if a teacher is assigned to more than one class at the same time.
    for day in state:
        for interval in state[day]:
            for classroom in state[day][interval]:

                if state[day][interval][classroom] is None:
                    continue

                teacher = state[day][interval][classroom][0]

                for other_classroom in state[day][interval]:
                    if other_classroom == classroom:
                        continue

                    if state[day][interval][other_classroom] is not None and state[day][interval][other_classroom][0] == teacher:
                        hard_cost += 1

    # Check if a subject cannot be taught in the assigned classroom.
    for day in state:
        for interval in state[day]:
            for classroom in state[day][interval]:
                if state[day][interval][classroom] is None:
                    continue
                
                subject = state[day][interval][classroom][1]

                if subject not in data[CLASSROOMS][classroom][SUBJECTS]:
                    hard_cost += 1
            
    return hard_cost

`check_soft_constraints_slot()` Calculates how many soft constraints have been violated in a specific time slot. 

In [ ]:
def check_soft_constraints_slot(state, day, interval, classroom, data):
    soft_cost = 0

    if state[day][interval][classroom] is None:
        return soft_cost

    teacher = state[day][interval][classroom][0]

    # Check if the teacher prefers the assigned interval.
    if interval in data[TEACHERS][teacher][NOT_PREFERRED]:
        soft_cost += 1

    # Check if the teacher prefers the assigned day.
    if day in data[TEACHERS][teacher][NOT_PREFERRED]:
        soft_cost += 1

    return soft_cost

`check_soft_constraints()` Calculates how many soft constraints have been violated for the timetable.

In [ ]:
def check_soft_constraints(state, data):
    soft_cost = 0
    
    for day in state:
        for interval in state[day]:
            for classroom in state[day][interval]:
                if state[day][interval][classroom] is None:
                    continue

                soft_cost += check_soft_constraints_slot(state, day, interval, classroom, data)

    return soft_cost

`is_final()` Checks if a state is final (if the state covers all subjects).

In [ ]:
def is_final(state, data):
    return all(is_covered(state, subject, data) for subject in data[SUBJECTS])

`generate_next_states()` Generates the neighbors of the current state.
    
Identify all empty slots in the timetable where a subject can be assigned.
For each empty slot, try assigning all possible combinations of teachers and subjects that satisfy the hard constraints.
Check each assignment to ensure that it doesn't violate any of the hard constraints.
If a valid assignment is found, create a new state representing the timetable with the assignment made.
Repeat this process for all empty slots, generating a list of all possible neighbor states.

In [ ]:
def generate_next_states(state, data):    
    next_states = []

    for subject in data[SUBJECTS]:

        # Check if this subject is covered.
        if is_covered(state, subject, data):
            continue

        for day in state:
            for interval in state[day]:
                for classroom in state[day][interval]:

                    # Empty slot is found.
                    if state[day][interval][classroom] is None:
                        new_state = deepcopy(state)

                        # Choose a teacher for this subject.
                        teacher = choose_teacher(state, subject, data)
                        
                        new_state[day][interval][classroom] = (teacher, subject)

                        # Verify hard constraints.
                        if check_hard_constraints(new_state, data) == 0:
                            next_states.append(new_state)

    return next_states

`count_unassigned_slots()` Counts how many time slots in the timetable have nothing assigned. \
`heuristic()` Heuristic function to evaluate a state. 

In [ ]:
def count_unassigned_slots(state):
    slots_count = 0
    for day in state:
        for interval in state[day]:
            for classroom in state[day][interval]:
                if state[day][interval][classroom] is None:
                    slots_count += 1
    
    return slots_count

def heuristic(state, data):
    return count_unassigned_slots(state) + check_soft_constraints(state, data)

`astar()` A* algorithm.

In [ ]:
def astar(initial_state, heuristic, generate_next_states, is_final, data):
    '''A* algorithm.'''

    #initial_state = generate_initial_state(initial_state, data)

    frontier = []
    heappush(frontier, State(heuristic(initial_state, data), initial_state))

    while frontier:
        current = heappop(frontier)

        current_state = current.state
        current_cost = current.cost

        if is_final(current_state, data):
            return current_state

        for next_state in generate_next_states(current_state, data):
            next_cost = heuristic(next_state, data)
            heappush(frontier, State(next_cost, next_state))
    

    return initial_state

`parse_input_file` Parses constraints from the input files. \
`run_astar()` Runs the A* algorithm for the given input file. \
`run_csp()` Runs the CSP algorithm for the given input file.


In [ ]:
def parse_input_file(input_file):
    data = utils.read_yaml_file(input_file)

    # Sort subjects by number of students.
    #data[SUBJECTS] = dict(sorted(data[SUBJECTS].items(), key=lambda item: item[1]))

    preferred = {}
    not_preferred = {}

    # Process the soft constraints for each teacher.
    for teacher in data[TEACHERS]:
        preferred = [constraint for constraint in data[TEACHERS][teacher][CONSTRAINTS] if not constraint.startswith('!')]
        not_preferred = [constraint.strip('!') for constraint in data[TEACHERS][teacher][CONSTRAINTS] if constraint.startswith('!')]


        # Convert intervals from '8-12' to (8, 10) and (10, 12).
        preferred_intervals = [tuple(map(int, interval.split('-'))) for interval in preferred if interval[0].isdigit()]
        not_preferred_intervals = [tuple(map(int, interval.split('-'))) for interval in not_preferred if interval[0].isdigit()]

        # Convert intervals from (8, 14) to [(8, 10), (10, 12), (12, 14)]. 
        preferred_intervals_expanded = []
        not_preferred_intervals_expanded = []

        for interval in preferred_intervals:
            start = interval[0]
            end = interval[1]

            if start != end - 2:
                intervals = [str((i, i + 2)) for i in range(start, end, 2)]
                preferred_intervals_expanded.extend(intervals)
            else:
                preferred_intervals_expanded.append(str(interval))

        for interval in not_preferred_intervals:
            start = interval[0]
            end = interval[1]

            if start != end - 2:
                intervals = [str((i, i + 2)) for i in range(start, end, 2)]
                not_preferred_intervals_expanded.extend(intervals)
            else:
                not_preferred_intervals_expanded.append(str(interval))
            

        # Create the preferred and not preferred days.
        preferred_days = [day for day in preferred if not day[0].isdigit()]
        not_preferred_days = [day.strip('!') for day in not_preferred if not day[0].isdigit()]
        
        data[TEACHERS][teacher][PREFERRED] = preferred_days + preferred_intervals_expanded
        data[TEACHERS][teacher][NOT_PREFERRED] = not_preferred_days + not_preferred_intervals_expanded

    return data

def run_astar(input_file):
    # Read input data.
    data = parse_input_file(input_file)

    # Initialize the empty timetable.
    initial_state = init_empty_timetable(data)

    # Run the A* algorithm.
    start = time.time()
    result = astar(initial_state, heuristic, generate_next_states, is_final, data)
    end = time.time()

    print(f'[A*] {input_file} Execution time: {round(end - start, 2)} seconds. Soft constraints violated: {check_soft_constraints(result, data)}')
    
    return result

def run_csp(input_file):
    data = parse_input_file(input_file)

    # Initialize the empty timetable.
    initial_state = init_empty_timetable(data)

`transform_intervals()` Function that transforms the intervals of the timetable from strings to tuples,
in order to work with the utils.pretty_print function.

In [ ]:
def transform_intervals(timetable):
    new_timetable = {}

    for day in timetable:
        new_timetable[day] = {}

        for interval in timetable[day]:
            temp = interval.strip('()').split(',')
            new_interval = (int(temp[0]), int(temp[1]))
            new_timetable[day][new_interval] = timetable[day][interval]

    return new_timetable                      

`test_astar()` Runs and times the A* algorithm for the input files. \
`test_csp()` Runs and times the CSP algorithm for the input files.

In [ ]:
input_files = ['dummy', 'orar_mic_exact', 'orar_mediu_relaxat', 'orar_mare_relaxat', 'orar_constrans_incalcat', 'orar_bonus_exact']

In [ ]:
def test_astar():
    for input_file in input_files:
        result = run_astar('inputs/' + input_file + '.yaml')
        transformed = transform_intervals(result)
        
        with open('outputs/' + input_file + '.txt', 'w') as f:
           f.write(utils.pretty_print_timetable(transformed, 'inputs/' + input_file + '.yaml'))

        print(utils.pretty_print_timetable(transformed, 'inputs/' + input_file + '.yaml'), end='\n')

def test_csp():
    for input_file in input_files:
        result = run_csp('inputs/' + input_file + '.yaml')
        transformed = transform_intervals(result)
        
        with open('outputs/' + input_file + '.txt', 'w') as f:
            f.write(utils.pretty_print_timetable(transformed, 'inputs/' + input_file + '.yaml'))

        print(utils.pretty_print_timetable(transformed, 'inputs/' + input_file + '.yaml'), end='\n')

test_astar()
test_csp()